In [ ]:
%matplotlib inline
import os
import sys
import gc
import time
from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from commons.configuration_manager import ConfigurationManager
from src.learning.training.collector import Collector
from src.learning.training.training_file_reader import TrainingFileReader
from src.utilities.transformer import Transformer
from src.learning.training.generator import Generator, GenFiles
from src.learning.models import create_cnn, create_mlp, create_multi_model
from notebooks.notebook_commons import read_stored_data, create_memorized_dataset

import pygame
import cv2

window_width = 500
window_height = 200

black = (0, 0, 0)
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)

pygame.init()
font = pygame.font.SysFont('Roboto', 20)

In [ ]:
def plot_stuff(title, plot_elems, figsize=(18, 10)):
    fig=plt.figure(figsize=figsize)
    plt.title(title)
    #plt.ylabel('dunno')
    plt.xlabel('Count')
    
    for plot_elem in plot_elems:
        plt.plot(plot_elem['data'], label=plot_elem['label'], alpha=plot_elem['alpha'], fmt='-o')

    plt.grid(axis='y')
    plt.legend(loc='best')
    plt.show()
    
    
def get_mem_dataset_with_full_video(filename, length, interval):
    mem_frames_np, mem_telemetry_np, mem_diffs_np = create_memorized_dataset(*read_stored_data(reader, transformer, filename, collector.steering_columns(), collector.diff_steering_columns()), length, interval)
    view_frames_np = reader.read_video(filename + '.avi')[((length - 1) * interval):]
    
    assert view_frames_np.shape[0] == mem_frames_np.shape[0], "Frames length mismatch!"
    assert mem_frames_np.shape[0] == mem_telemetry_np.shape[0] == mem_diffs_np.shape[0], "Mem lengths differ!"
    return mem_frames_np, mem_telemetry_np, mem_diffs_np, view_frames_np


def validate_partial(memory, i):
    memory_string = 'n{}_m{}'.format(*memory)
    path = '../../training/' + memory_string + '/'

    mem_frame = np.load(path + GenFiles.frame.format(memory_string, i)) * 255
    return mem_frame.astype(np.uint8)[:,:,:3]

In [ ]:
config_manager = ConfigurationManager()
config = config_manager.config

reader = TrainingFileReader(path_to_training='../../training/laps/')
collector = Collector()

filenames = ['lap_5_2020_01_24', 'lap_6_2020_01_24', 'lap_7_2020_01_24']
experiments = [(1, 1), (4, 1), (4, 4), (8, 1), (8, 4), (16, 1)]
memory = experiments[4]

epochs = 10
batch_size = 64
verbose = 1

transformer = Transformer(config, memory_tuple=memory)
generator = Generator(config, memory_tuple=memory, base_path='../../training/', batch_size=batch_size, column_mode='steer', separate_files=True)

mem_frames, mem_telems, mem_diffs, display_frames = get_mem_dataset_with_full_video(filenames[2], *memory)

mlp = create_mlp(input_shape=mem_telems[0].shape)
cnn = create_cnn(input_shape=mem_frames[0].shape)
multi = create_multi_model(mlp, cnn, output_shape=mem_diffs[0].shape[0])

multi.fit(generator.generate(data='train'),
          steps_per_epoch=generator.train_batch_count,
          validation_data=generator.generate(data='test'),
          validation_steps=generator.test_batch_count,
          epochs=epochs, verbose=verbose)

In [ ]:
pygame.display.init()
pygame.display.set_caption("Prediction viewer")
screen = pygame.display.set_mode((window_width, window_height))

try:
    for i in range(0, display_frames.shape[0]):
        frame = display_frames[i]
        screen.fill(black)
        
        converted_telem = mem_telems[i][np.newaxis, :]
        converted_frame = mem_frames[i].astype(np.float32)[np.newaxis, :]
        
        # gear, steering, throttle, braking
        predictions = multi.predict([converted_frame, converted_telem])[0]
        
        predicted_steering = predictions[0]
        prediction_text = "{0:.4f}".format(predicted_steering)
        if predicted_steering >= 0.0:
            prediction_text = font.render(prediction_text, True, green)
        else:
            prediction_text = font.render(prediction_text, True, blue)
        
        actual_steering = "{}".format(mem_diffs[i])
        if mem_diffs[i] >= 0.0:
            actual_text = font.render(actual_steering, True, green)
        else:
            actual_text = font.render(actual_steering, True, blue)
        
        #frame = mem_frames[i][:,:,:3]
        #frame = validate_partial(memory, i)
        frame = np.rot90(frame)
        surface = pygame.surfarray.make_surface(frame)
        
        x = (window_width - frame.shape[0]) // 2
        y = 0
        
        time.sleep(0.05)
        screen.blit(prediction_text, (200, window_height - 75))
        screen.blit(actual_text, (200, window_height - 45))
        screen.blit(surface, (x, y))
        pygame.display.update()

        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    sys.exit(0)
    pygame.display.quit()
except:
    pygame.display.quit()

In [ ]:
from tensorflow.keras.utils import plot_model

model_path = '../../training/models/'
model_file_prefix = 'model_n{}_m{}'.format(*memory)
model_file_suffix = '_{}.{}'

count = len([fn for fn in os.listdir(model_path) if fn.startswith(model_file_prefix)])

plot_model(multi, to_file=model_path + model_file_prefix + model_file_suffix.format(count + 1, 'png'))
multi.save(model_path + model_file_prefix + model_file_suffix.format(count + 1, 'h5'))